In [ ]:
import pickle
import time
start_time = time.time()
import numpy as np
import os
import SimpleITK as sitk
import json
import ProstateLesionDetectionUtils
import random
import plotly.express as px
import matplotlib.pyplot as plt
from scipy import ndimage
import math
import tensorflow_addons as tfa
import tensorflow as tf
with open(r"E:\DIMITRIS_DEEP_LEARNING\DATASETS\UC1_Nifti_Datasets_Paths\Datasets.json", 'r') as f:
    datasets = json.load(f)
p158 = r"E:\DIMITRIS_DEEP_LEARNING\DATASETS\PROSTATE158\DATASETPICKLE\Nifti_files"
picai = r"E:\DIMITRIS_DEEP_LEARNING\DATASETS\PICAI_DATA\Nifti_files"
prostatenet = r"E:\DIMITRIS_DEEP_LEARNING\DATASETS\PROSTATENET_UC1_LESIONS\nnUNet_raw\Dataset011_ProstateNet"
with open(r"E:\DIMITRIS_DEEP_LEARNING\DATASETS\PROSTATENET_UC1_LESIONS\nnUNet_raw\Dataset011_ProstateNet\folds.json", 'r') as f:
    ValidationFolds = json.load(f)
    

def PrepareTraining(pat):
    dataset, labels = {}, {}
    for key in pat.keys():
        dataset.update({key:np.transpose(np.stack([pat[key]["T2"], pat[key]["ADC"], pat[key]["DWI"]]),(1,2,3,0))}) # tf.convert_to_tensor
        labels.update({key:np.expand_dims(pat[key]["Lesions"], axis = -1)}) # tf.convert_to_tensor
    return dataset, labels

In [ ]:
loader = ProstateLesionDetectionUtils.DatasetsLoaders.Nifti_Loader.LoadSitkImages(ptdc = datasets["ProstateNet"], folder = prostatenet)
patients = loader.load_npobj()

In [ ]:
dataset, labels = PrepareTraining(patients)

x_train = {key:dataset[key] for key in dataset.keys() if key not in ValidationFolds["Fold 0"]}
y_train = {key:labels[key] for key in labels.keys() if key not in ValidationFolds["Fold 0"]}

x_test = {key:dataset[key] for key in dataset.keys() if key in ValidationFolds["Fold 0"]}
y_test = {key:labels[key] for key in labels.keys() if key in ValidationFolds["Fold 0"]}